In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report, confusion_matrix
from tabulate import tabulate
import time
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [6]:
param_grid_boosting = {
    "n_estimators": range(1,101),  
    "learning_rate": [0.1, 0.5], 
}

# 1 sekundowe pliki

## 13 cech

In [7]:
train_data = pd.read_csv("Sounds_1_seconds/mfcc_1_sec/train_13_mfcc_features_with_labels_1sec.csv")  
test_data = pd.read_csv("Sounds_1_seconds/mfcc_1_sec/test_13_mfcc_features_with_labels_1sec.csv")    

X_train = train_data.drop(['Encoded_label'], axis=1) 
y_train = train_data['Encoded_label']  

X_test = test_data.drop(['Encoded_label'], axis=1)  
y_test = test_data['Encoded_label']

In [8]:
results_boosting = []

for n_estimators in param_grid_boosting["n_estimators"]:
    for learning_rate in param_grid_boosting["learning_rate"]:
        try:
            
            boosting_model = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                random_state=42
            )
            
            start_time = time.time()
            boosting_model.fit(X_train, y_train)
            training_time = time.time() - start_time
            
            y_pred_boosting = boosting_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred_boosting)
            f1 = f1_score(y_test, y_pred_boosting, average='weighted')
            recall = recall_score(y_test, y_pred_boosting, average='weighted')
            precision = precision_score(y_test, y_pred_boosting, average='weighted', zero_division=0)

            results_boosting.append({
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "Accuracy": accuracy,
                "F1 Score": f1,
                "Recall": recall,
                "Precision": precision,
                "Training Time (s)": training_time
            })
        except Exception as e:
            print(f"Error with n_estimators={n_estimators}, learning_rate={learning_rate}: {e}")

metrics_boosting = ["Accuracy", "F1 Score", "Recall", "Precision"]
best_boosting_models = {metric: max(results_boosting, key=lambda x: x[metric]) for metric in metrics_boosting}
best_boosting_models["Training Time"] = min(results_boosting, key=lambda x: x["Training Time (s)"])

all_models_boosting_table = [
    [res["n_estimators"], res["learning_rate"], res["Accuracy"], res["F1 Score"], 
     res["Recall"], res["Precision"], res["Training Time (s)"]]
    for res in results_boosting
]

best_models_boosting_table = [
    [metric, 
     best_boosting_models[metric]["n_estimators"], 
     best_boosting_models[metric]["learning_rate"], 
     best_boosting_models[metric]["Accuracy"], 
     best_boosting_models[metric]["F1 Score"], 
     best_boosting_models[metric]["Recall"], 
     best_boosting_models[metric]["Precision"], 
     best_boosting_models[metric]["Training Time (s)"]]
    for metric in metrics_boosting + ["Training Time"]
]

print("Best Models (Boosting):")
print(tabulate(best_models_boosting_table, headers=[
    "Metric", "n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

Best Models for Each Metric (Boosting):
╒═══════════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╤═════════════════════╕
│ Metric        │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │   Training Time (s) │
╞═══════════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╪═════════════════════╡
│ Accuracy      │             98 │             0.5 │   0.929819 │   0.929733 │ 0.929819 │    0.929692 │             51.9501 │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ F1 Score      │             98 │             0.5 │   0.929819 │   0.929733 │ 0.929819 │    0.929692 │             51.9501 │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ Recall        │             98 │             0.5 │   0.929819 │   0.929733 │

In [9]:
model = GradientBoostingClassifier(
                n_estimators=98,
                learning_rate=0.5,
                random_state=42
            )

start_time = time.time()
model.fit(X_train, y_train)
training_time = time.time() - start_time

unique_labels = y_test.unique()
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average = 'weighted')

print(" Boosting:")
print(f" Dokładność (Accuracy): {accuracy:.4f}")
print(f" Precyzja (Precision): {precision:.4f}")
print(f" Recall: {recall:.4f}")
print(f" F1 score: {f1:.4f}")
print(f" Czas treningu: {training_time:.4f} sekund")
print("\nClassification Report:")


print(classification_report(y_test, y_pred, target_names=[str(label) for label in sorted(unique_labels)]))

label_mapping = {
    0: "Bęben",
    1: "Gitara",
    2: "Fortepian",
    3: "Skrzypce"
}

sorted_labels = np.sort(unique_labels)
mapped_labels = [label_mapping[label] for label in sorted_labels]
conf_matrix = confusion_matrix(y_test, y_pred, labels=sorted_labels)

plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt=".1f", cmap="Greys", 
            xticklabels=mapped_labels, yticklabels=mapped_labels, cbar_kws={'label': 'Liczba przewidywań'})

plt.xlabel("Przewidywane", fontsize=14)
plt.ylabel("Rzeczywiste", fontsize=14)
plt.title("Macierz pomyłek dla drzewa decyzyjnego", fontsize=16, fontweight='bold')
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12, rotation=0)
plt.tight_layout()
plt.show()


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [10]:
print(tabulate(all_models_boosting_table, headers=[
"n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

╒══════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╕
│   Metric │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │
╞══════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╡
│        1 │            0.1 │        0.829518 │   0.830011 │   0.829518 │ 0.831509 │     0.56214 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        1 │            0.5 │        0.829518 │   0.830011 │   0.829518 │ 0.831509 │     0.5553  │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.1 │        0.82259  │   0.820456 │   0.82259  │ 0.81943  │     1.11222 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.5 │        0.848795 │   0.848463 │   0.848795 │ 0.848243 │     1.08182 │
├─────────

## 30 cech

In [11]:
train_data = pd.read_csv("Sounds_1_seconds/mfcc_1_sec/train_30_mfcc_features_with_labels_1sec.csv")  
test_data = pd.read_csv("Sounds_1_seconds/mfcc_1_sec/test_30_mfcc_features_with_labels_1sec.csv")    

X_train = train_data.drop(['Encoded_label'], axis=1) 
y_train = train_data['Encoded_label']  

X_test = test_data.drop(['Encoded_label'], axis=1)  
y_test = test_data['Encoded_label']

In [12]:
results_boosting = []

for n_estimators in param_grid_boosting["n_estimators"]:
    for learning_rate in param_grid_boosting["learning_rate"]:
        try:
            
            boosting_model = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                random_state=42
            )
            
            start_time = time.time()
            boosting_model.fit(X_train, y_train)
            training_time = time.time() - start_time
            
            y_pred_boosting = boosting_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred_boosting)
            f1 = f1_score(y_test, y_pred_boosting, average='weighted')
            recall = recall_score(y_test, y_pred_boosting, average='weighted')
            precision = precision_score(y_test, y_pred_boosting, average='weighted', zero_division=0)

            results_boosting.append({
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "Accuracy": accuracy,
                "F1 Score": f1,
                "Recall": recall,
                "Precision": precision,
                "Training Time (s)": training_time
            })
        except Exception as e:
            print(f"Error with n_estimators={n_estimators}, learning_rate={learning_rate}: {e}")

metrics_boosting = ["Accuracy", "F1 Score", "Recall", "Precision"]
best_boosting_models = {metric: max(results_boosting, key=lambda x: x[metric]) for metric in metrics_boosting}
best_boosting_models["Training Time"] = min(results_boosting, key=lambda x: x["Training Time (s)"])

all_models_boosting_table = [
    [res["n_estimators"], res["learning_rate"], res["Accuracy"], res["F1 Score"], 
     res["Recall"], res["Precision"], res["Training Time (s)"]]
    for res in results_boosting
]

best_models_boosting_table = [
    [metric, 
     best_boosting_models[metric]["n_estimators"], 
     best_boosting_models[metric]["learning_rate"], 
     best_boosting_models[metric]["Accuracy"], 
     best_boosting_models[metric]["F1 Score"], 
     best_boosting_models[metric]["Recall"], 
     best_boosting_models[metric]["Precision"], 
     best_boosting_models[metric]["Training Time (s)"]]
    for metric in metrics_boosting + ["Training Time"]
]

print("Best Models (Boosting):")
print(tabulate(best_models_boosting_table, headers=[
    "Metric", "n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

Best Models for Each Metric (Boosting):
╒═══════════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╤═════════════════════╕
│ Metric        │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │   Training Time (s) │
╞═══════════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╪═════════════════════╡
│ Accuracy      │             94 │             0.5 │   0.952259 │   0.952215 │ 0.952259 │    0.952183 │           112.539   │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ F1 Score      │             94 │             0.5 │   0.952259 │   0.952215 │ 0.952259 │    0.952183 │           112.539   │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ Recall        │             94 │             0.5 │   0.952259 │   0.952215 │

In [13]:
print(tabulate(all_models_boosting_table, headers=[
"n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

╒══════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╕
│   Metric │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │
╞══════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╡
│        1 │            0.1 │        0.831325 │   0.832226 │   0.831325 │ 0.834031 │     1.23029 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        1 │            0.5 │        0.831325 │   0.832226 │   0.831325 │ 0.834031 │     1.21827 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.1 │        0.821084 │   0.820646 │   0.821084 │ 0.820359 │     2.41954 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.5 │        0.854367 │   0.853935 │   0.854367 │ 0.85386  │     2.42153 │
├─────────

## 50 cech

In [14]:
train_data = pd.read_csv("Sounds_1_seconds/mfcc_1_sec/train_50_mfcc_features_with_labels_1sec.csv")  
test_data = pd.read_csv("Sounds_1_seconds/mfcc_1_sec/test_50_mfcc_features_with_labels_1sec.csv")    

X_train = train_data.drop(['Encoded_label'], axis=1) 
y_train = train_data['Encoded_label']  

X_test = test_data.drop(['Encoded_label'], axis=1)  
y_test = test_data['Encoded_label']

In [15]:

results_boosting = []

for n_estimators in param_grid_boosting["n_estimators"]:
    for learning_rate in param_grid_boosting["learning_rate"]:
        try:
            
            boosting_model = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                random_state=42
            )
            
            start_time = time.time()
            boosting_model.fit(X_train, y_train)
            training_time = time.time() - start_time
            
            y_pred_boosting = boosting_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred_boosting)
            f1 = f1_score(y_test, y_pred_boosting, average='weighted')
            recall = recall_score(y_test, y_pred_boosting, average='weighted')
            precision = precision_score(y_test, y_pred_boosting, average='weighted', zero_division=0)

            results_boosting.append({
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "Accuracy": accuracy,
                "F1 Score": f1,
                "Recall": recall,
                "Precision": precision,
                "Training Time (s)": training_time
            })
        except Exception as e:
            print(f"Error with n_estimators={n_estimators}, learning_rate={learning_rate}: {e}")

metrics_boosting = ["Accuracy", "F1 Score", "Recall", "Precision"]
best_boosting_models = {metric: max(results_boosting, key=lambda x: x[metric]) for metric in metrics_boosting}
best_boosting_models["Training Time"] = min(results_boosting, key=lambda x: x["Training Time (s)"])

all_models_boosting_table = [
    [res["n_estimators"], res["learning_rate"], res["Accuracy"], res["F1 Score"], 
     res["Recall"], res["Precision"], res["Training Time (s)"]]
    for res in results_boosting
]

best_models_boosting_table = [
    [metric, 
     best_boosting_models[metric]["n_estimators"], 
     best_boosting_models[metric]["learning_rate"], 
     best_boosting_models[metric]["Accuracy"], 
     best_boosting_models[metric]["F1 Score"], 
     best_boosting_models[metric]["Recall"], 
     best_boosting_models[metric]["Precision"], 
     best_boosting_models[metric]["Training Time (s)"]]
    for metric in metrics_boosting + ["Training Time"]
]

print("Best Models (Boosting):")
print(tabulate(best_models_boosting_table, headers=[
    "Metric", "n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

Best Models for Each Metric (Boosting):
╒═══════════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╤═════════════════════╕
│ Metric        │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │   Training Time (s) │
╞═══════════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╪═════════════════════╡
│ Accuracy      │             99 │             0.5 │   0.956777 │   0.956771 │ 0.956777 │    0.956766 │           198.88    │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ F1 Score      │             99 │             0.5 │   0.956777 │   0.956771 │ 0.956777 │    0.956766 │           198.88    │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ Recall        │             99 │             0.5 │   0.956777 │   0.956771 │

In [16]:
print(tabulate(all_models_boosting_table, headers=[
"n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

╒══════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╕
│   Metric │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │
╞══════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╡
│        1 │            0.1 │        0.831325 │   0.832226 │   0.831325 │ 0.834031 │     2.01245 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        1 │            0.5 │        0.831325 │   0.832226 │   0.831325 │ 0.834031 │     1.98344 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.1 │        0.821536 │   0.821043 │   0.821536 │ 0.820733 │     3.94587 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.5 │        0.854367 │   0.853972 │   0.854367 │ 0.853919 │     3.97087 │
├─────────

# 3 sekundowe pliki

In [17]:
train_data = pd.read_csv("Sounds_3_seconds/mfcc_3_sec/train_13_mfcc_features_with_labels_3sec.csv")  
test_data = pd.read_csv("Sounds_3_seconds/mfcc_3_sec/test_13_mfcc_features_with_labels_3sec.csv")    

X_train = train_data.drop(['Encoded_label'], axis=1) 
y_train = train_data['Encoded_label']  

X_test = test_data.drop(['Encoded_label'], axis=1)  
y_test = test_data['Encoded_label']

In [18]:
results_boosting = []

for n_estimators in param_grid_boosting["n_estimators"]:
    for learning_rate in param_grid_boosting["learning_rate"]:
        try:
            
            boosting_model = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                random_state=42
            )
            
            start_time = time.time()
            boosting_model.fit(X_train, y_train)
            training_time = time.time() - start_time
            
            y_pred_boosting = boosting_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred_boosting)
            f1 = f1_score(y_test, y_pred_boosting, average='weighted')
            recall = recall_score(y_test, y_pred_boosting, average='weighted')
            precision = precision_score(y_test, y_pred_boosting, average='weighted', zero_division=0)

            results_boosting.append({
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "Accuracy": accuracy,
                "F1 Score": f1,
                "Recall": recall,
                "Precision": precision,
                "Training Time (s)": training_time
            })
        except Exception as e:
            print(f"Error with n_estimators={n_estimators}, learning_rate={learning_rate}: {e}")

metrics_boosting = ["Accuracy", "F1 Score", "Recall", "Precision"]
best_boosting_models = {metric: max(results_boosting, key=lambda x: x[metric]) for metric in metrics_boosting}
best_boosting_models["Training Time"] = min(results_boosting, key=lambda x: x["Training Time (s)"])

all_models_boosting_table = [
    [res["n_estimators"], res["learning_rate"], res["Accuracy"], res["F1 Score"], 
     res["Recall"], res["Precision"], res["Training Time (s)"]]
    for res in results_boosting
]

best_models_boosting_table = [
    [metric, 
     best_boosting_models[metric]["n_estimators"], 
     best_boosting_models[metric]["learning_rate"], 
     best_boosting_models[metric]["Accuracy"], 
     best_boosting_models[metric]["F1 Score"], 
     best_boosting_models[metric]["Recall"], 
     best_boosting_models[metric]["Precision"], 
     best_boosting_models[metric]["Training Time (s)"]]
    for metric in metrics_boosting + ["Training Time"]
]

print("Best Models (Boosting):")
print(tabulate(best_models_boosting_table, headers=[
    "Metric", "n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

Best Models for Each Metric (Boosting):
╒═══════════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╤═════════════════════╕
│ Metric        │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │   Training Time (s) │
╞═══════════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╪═════════════════════╡
│ Accuracy      │             98 │             0.5 │   0.948276 │   0.948342 │ 0.948276 │    0.948923 │            17.1525  │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ F1 Score      │             98 │             0.5 │   0.948276 │   0.948342 │ 0.948276 │    0.948923 │            17.1525  │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ Recall        │             98 │             0.5 │   0.948276 │   0.948342 │

In [19]:
print(tabulate(all_models_boosting_table, headers=[
"n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

╒══════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╕
│   Metric │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │
╞══════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╡
│        1 │            0.1 │        0.835345 │   0.831211 │   0.835345 │ 0.83418  │    0.178046 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        1 │            0.5 │        0.835345 │   0.831211 │   0.835345 │ 0.83418  │    0.17504  │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.1 │        0.846121 │   0.841587 │   0.846121 │ 0.846296 │    0.350073 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.5 │        0.849138 │   0.845995 │   0.849138 │ 0.848601 │    0.348084 │
├─────────

# 5 sekundowe pliki

In [22]:
train_data = pd.read_csv("Sounds_5_seconds/mfcc_5_sec/train_13_mfcc_features_with_labels_5sec.csv")  
test_data = pd.read_csv("Sounds_5_seconds/mfcc_5_sec/test_13_mfcc_features_with_labels_5sec.csv")    

X_train = train_data.drop(['Encoded_label'], axis=1) 
y_train = train_data['Encoded_label']  

X_test = test_data.drop(['Encoded_label'], axis=1)  
y_test = test_data['Encoded_label']

In [23]:
results_boosting = []

for n_estimators in param_grid_boosting["n_estimators"]:
    for learning_rate in param_grid_boosting["learning_rate"]:
        try:
            
            boosting_model = GradientBoostingClassifier(
                n_estimators=n_estimators,
                learning_rate=learning_rate,
                random_state=42
            )
            
            start_time = time.time()
            boosting_model.fit(X_train, y_train)
            training_time = time.time() - start_time
            
            y_pred_boosting = boosting_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred_boosting)
            f1 = f1_score(y_test, y_pred_boosting, average='weighted')
            recall = recall_score(y_test, y_pred_boosting, average='weighted')
            precision = precision_score(y_test, y_pred_boosting, average='weighted', zero_division=0)

            results_boosting.append({
                "n_estimators": n_estimators,
                "learning_rate": learning_rate,
                "Accuracy": accuracy,
                "F1 Score": f1,
                "Recall": recall,
                "Precision": precision,
                "Training Time (s)": training_time
            })
        except Exception as e:
            print(f"Error with n_estimators={n_estimators}, learning_rate={learning_rate}: {e}")

metrics_boosting = ["Accuracy", "F1 Score", "Recall", "Precision"]
best_boosting_models = {metric: max(results_boosting, key=lambda x: x[metric]) for metric in metrics_boosting}
best_boosting_models["Training Time"] = min(results_boosting, key=lambda x: x["Training Time (s)"])

all_models_boosting_table = [
    [res["n_estimators"], res["learning_rate"], res["Accuracy"], res["F1 Score"], 
     res["Recall"], res["Precision"], res["Training Time (s)"]]
    for res in results_boosting
]

best_models_boosting_table = [
    [metric, 
     best_boosting_models[metric]["n_estimators"], 
     best_boosting_models[metric]["learning_rate"], 
     best_boosting_models[metric]["Accuracy"], 
     best_boosting_models[metric]["F1 Score"], 
     best_boosting_models[metric]["Recall"], 
     best_boosting_models[metric]["Precision"], 
     best_boosting_models[metric]["Training Time (s)"]]
    for metric in metrics_boosting + ["Training Time"]
]

print("Best Models (Boosting):")
print(tabulate(best_models_boosting_table, headers=[
    "Metric", "n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

Best Models for Each Metric (Boosting):
╒═══════════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╤═════════════════════╕
│ Metric        │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │   Training Time (s) │
╞═══════════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╪═════════════════════╡
│ Accuracy      │             86 │             0.5 │   0.953125 │   0.953084 │ 0.953125 │    0.95323  │           7.97677   │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ F1 Score      │             91 │             0.5 │   0.953125 │   0.953108 │ 0.953125 │    0.953226 │           8.44889   │
├───────────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┼─────────────────────┤
│ Recall        │             86 │             0.5 │   0.953125 │   0.953084 │

In [24]:
print(tabulate(all_models_boosting_table, headers=[
"n_estimators", "learning_rate", "Accuracy", "F1 Score", "Recall", "Precision", "Training Time (s)"
], tablefmt="fancy_grid"))

╒══════════╤════════════════╤═════════════════╤════════════╤════════════╤══════════╤═════════════╕
│   Metric │   n_estimators │   learning_rate │   Accuracy │   F1 Score │   Recall │   Precision │
╞══════════╪════════════════╪═════════════════╪════════════╪════════════╪══════════╪═════════════╡
│        1 │            0.1 │        0.849219 │   0.846534 │   0.849219 │ 0.848479 │   0.0960197 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        1 │            0.5 │        0.849219 │   0.846534 │   0.849219 │ 0.848479 │   0.0930202 │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.1 │        0.853125 │   0.850323 │   0.853125 │ 0.852628 │   0.185045  │
├──────────┼────────────────┼─────────────────┼────────────┼────────────┼──────────┼─────────────┤
│        2 │            0.5 │        0.857812 │   0.856066 │   0.857812 │ 0.857454 │   0.191052  │
├─────────